In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 25 20:13:34 2019

@author: FinalZ
"""
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 25 20:13:34 2019

@author: FinalZ
"""
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 25 20:13:34 2019
@author: FinalZ
"""

from calculate_genotype_k_mer import queryKmer
import pandas as pd
import imblearn
# print(imblearn.__version__)
from imblearn.ensemble import BalancedBaggingClassifier
# help(BalancedBaggingClassifier)
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import resample
import getopt,sys
import time

In [ ]:
start_time = time.time()


#python predResult.py --query test --output predict_result --file mod_data --kmer 4 --kjob 1 --bjob 1
opts, args = getopt.getopt(sys.argv[1:], "hi:o:",["query=","output=","file=","kmer=","kjob=","bjob="])
for op, value in opts:
    if op == "--query":
        query = value
    elif op == "--output":
        output = value
    elif op == "--file":
        infile = value
    elif op == "--kmer":
        kmer = int(value)
    elif op == "--kjob":
        k_job = int(value)
    elif op == "--bjob":
        b_job = int(value)


In [ ]:
###########################################################
#classifier query sequences by different length
# test_500, test_1000,test_3000,test_5000,test_10000,test_genome = [],[],[],[],[],[]
# Initialization of lists for different sequence lengths
test_500 = []
test_1000 = []
test_3000 = []
test_5000 = []
test_10000 = []
test_genome = []


#file = open(query,'r')
query_name = []
#sequences = parse_fasta(query)

name = ""  # Default or initial value for 'name'
file = open(query, 'r')
for line in file:
    line = line.strip('\n')
    if line.startswith('NC'):  # Condition to check and set 'name'
        name = line
        continue
    if len(line) < 1000:
        test_500.append(name+' '+line)
    elif len(line) < 3000:
        test_1000.append(name+' '+line)
    elif len(line) < 5000:
        test_3000.append(name+' '+line)
    elif len(line) < 10000:
        test_5000.append(name+' '+line)
    elif len(line) < 15000:
        test_10000.append(name+' '+line)
    else:
        test_genome.append(name+' '+line)
file.close()

In [ ]:
result = pd.DataFrame({"name":query_name})
result.index = result.iloc[:,0]

###########################################################
lengths = [test_500,test_1000,test_3000,test_5000,test_10000,test_genome]
piece = ['500bp','1000bp','3000bp','5000bp','10000bp','genome']
# pred_result = []
# isFirst = True
# for ll in range(len(lengths)):
#     if lengths[ll] == []:
#         continue

ll = 4 #arbitrary number in range. Because of breaking we are no longer running this as a loop

In [ ]:
########################################
data = pd.read_csv('./'+infile+'/'+piece[ll]+'_'+str(kmer)+'_mer',sep = '\t',header = None,index_col = False)
X = data.iloc[:,1:(data.shape[1] - 1)]#get fearture
y = data.iloc[:,data.shape[1] - 1]#get label
del data
X_train = np.array(X)
y_train = np.array(y)
del X,y#clear useless variable, Free memory

#Base Estimater
knn = KNeighborsClassifier(n_neighbors=1, n_jobs=k_job)
#set model parameter
model = BalancedBaggingClassifier(estimator=knn, n_estimators =10, n_jobs = b_job)
#training the model
model.fit(X_train,y_train)

#clear useless variable, Free memory
del X_train,y_train

print(''+piece[ll]+' model ready')
#########################################

In [ ]:
#predict query sequences

#In order to reduce memory usage, we only calculate 1000 sequences at a time.
step = 1000
sub_query = [lengths[ll][i:i + step] for i in range(0, len(lengths[ll]), step)]
# Filter out sequences that are too short before processing
# Ensure sub_query is defined and populated correctly first
# sub_query = [' '.join(s.split(' ')[0:2]) for s in sub_query for sub in s if len(sub.split(' ')[1]) >= kmer]
flat_sub_query = [item for sublist in sub_query for item in sublist]

# Now, filter out sequences that are too short to calculate k-mers
filtered_sub_query = [entry for entry in flat_sub_query if ' ' in entry and len(entry.split(' ')) >= 2]

X_test = np.empty(256)
for identifier, seq in zip(filtered_sub_query, filtered_sub_query[1:]):
    if 'ATG' in seq:
        data = queryKmer(identifier, seq, kmer)
    else:
        continue

    #get query sequence fearture
    X_test = np.vstack([X_test,np.array(data.iloc[:,1:])])

print(X_test.shape)
y_pred = np.array(model.predict(X_test))
predict_prob_y = np.array(model.predict_proba(X_test)[:,1])
pred_data = pd.DataFrame(
    {
    'Model' : piece[ll],
    'Label' : y_pred,
    'Probability' : predict_prob_y
    }, 
    index = [data.iloc[:,0].values[0] for i in range(len(y_pred))]
)

if isFirst:
    isFirst = False
    pred_result = pred_data
else:#append predict result
    pred_result = pred_result._append(pred_data)


In [ ]:
#clear useless variable, Free memory
print("Data shape:", X_test.shape)
y_pred = model.predict(X_test)
predict_prob_y = model.predict_proba(X_test)[:, 1]  # assuming binary classification

print("Sample predictions:", y_pred[:5])
print("Sample probabilities:", predict_prob_y[:5])

# Ensure DataFrame is being updated
pred_data['Label'] = y_pred
pred_data['Probability'] = predict_prob_y
print("Preview of prediction DataFrame:", pred_data.head())
del data, X_test
    

In [ ]:
#print the predict result
result = result.join(pred_result, how='outer')
result.drop("name",axis=1, inplace=True)
result.index.name = 'Name'
result.to_csv(''+output+'',sep = '\t',header = True,index = True)

end_time = time.time()
total_time = end_time - start_time
print ('Total running time of the program: %.2f seconds' % (total_time))
#clear useless variable, Free memory
del pred_result, result,test_500,test_1000,test_3000,test_5000,test_10000,test_genome,lengths,model